In [103]:
def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input() -> list[str]:
    return get_lines_from_file('./test_input')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func

In [104]:
from typing import NamedTuple
from enum import Enum
import re

CMD = Enum('CMD', ['ADDX', 'NOOP'])
Command = NamedTuple('Command', [('type', CMD), ('value', int)])

NOOP = re.compile(r'noop')
ADDX = re.compile(r'addx (-?\d+)')

class CPU:
    def __init__(self, register_init: int) -> None:
        self._register = register_init
        self._pc = 0
        self._command_cycles = 0
        self._program: list[Command] = []
        self._cycle = 0

    @property
    def cycle(self):
        return self._cycle

    @property
    def program(self):
        return self._program

    @program.setter
    def program(self, program: list[Command]):
        self._program = program
        self._pc = 0

    @property
    def register(self) -> int:
        return self._register

    def tick(self):
        command = self._program[self._pc]
        self._cycle += 1
        if command.type == CMD.ADDX:
            if self._command_cycles == 0:
                self._command_cycles = 1
            elif self._command_cycles == 1:
                self._register += command.value
                self._command_cycles = 0
                self._pc += 1
        elif command.type == CMD.NOOP:
            self._pc += 1

    def multi_tick(self, count: int):
        for _ in range(count):
            self.tick()

def parse_command(input: str):
    if NOOP.match(input) is not None:
        return Command(CMD.NOOP, 0)
    if (m := ADDX.match(input)) is not None:
        return Command(CMD.ADDX, int(m.group(1)))

def solution1(input: list[str]) -> int:
    interesting_cycles = [19, 59, 99, 139, 179, 219]

    commands = [parse_command(line) for line in input]

    cpu = CPU(1)
    cpu.program = commands

    current_cycle = 0
    signal_strengths = []

    for cycle in interesting_cycles:
        cpu.multi_tick(cycle - current_cycle)
        current_cycle = cycle
        signal_strengths.append((cycle+1) * cpu.register)
    
    return sum(signal_strengths)


In [105]:
def is_pixel_active(sprite_pos: int, render_pos: int):
    return render_pos >= sprite_pos-1 and render_pos <= sprite_pos+1

def render_screen(screen: list[str]):
    for i in range(len(screen) // 40):
        print(screen[i*40:(i+1)*40])

def solution2(input: list[str]) -> int:
    commands = [parse_command(line) for line in input]

    cpu = CPU(1)
    cpu.program = commands

    screen = ''

    for i in range(240):
        if is_pixel_active(cpu.register, i%40):
            screen += '#'
        else:
            screen += '.'

        cpu.tick()

    render_screen(screen)
    
    return -1

In [106]:
solutions = [
    solution1,
    solution2,
]

test_results = [
    get_int_from_file('./test_result1'),
    get_int_from_file('./test_result2'),
]

def run_test(idx) -> bool:
    res = solutions[idx-1](get_test_input())
    test_res = test_results[idx-1]
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx):
    sol = solutions[idx-1](get_input())
    print(f'The solution for part {idx} is: {sol}')

if run_test(1):
    run_solution(1)
    print('\nOn to part 2...\n')
    if run_test(2):
        run_solution(2)

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 14040

On to part 2...

##..##..##..##..##..##..##..##..##..##..
###...###...###...###...###...###...###.
####....####....####....####....####....
#####.....#####.....#####.....#####.....
######......######......######......####
#######.......#######.......#######.....
Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
####..##...##....##.####...##.####.#....
...#.#..#.#..#....#....#....#.#....#....
..#..#....#.......#...#.....#.###..#....
.#...#.##.#.......#..#......#.#....#....
#....#..#.#..#.#..#.#....#..#.#....#....
####..###..##...##..####..##..#....####.
The solution for part 2 is: -1
